<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65


    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 13:07:29
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.37 C
-------------------
Today PnL: 49.37 K (0.36%)
Current PnL: -20.34 L (-13.68%)
CY Booked + Current PnL: -11.30 L (-7.6%)
-------------------
Total profit:  2.71 L
Total loss:  -23.04 L
-------------------
Total Booked + Current PnL: 15.68 L (12.89%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.6%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 81.72 L (59.69%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,55.0,H-LC,7.47,112224.0,7912.0,8742.0,-0.61,7.58,7.79,15.96,9.0,0.91,0.85,25.76,X5K,ATH,METALS
53,MEDANTA,1486.00,13.12,59.0,H-MC,15.05,161414.0,35214.0,10960.0,-0.60,27.90,6.79,36.59,108.0,3.21,1.22,43.97,XY24,NTT,HEALTHCARE
81,TTKPRESTIG,770.00,100.43,61.0,M-SC,8.16,86329.0,-14448.0,14538.0,1.82,-14.34,16.84,0.09,245.0,-0.99,0.65,13.42,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-16.30,51.0,H-LC,8.37,167178.0,11001.0,15096.0,-1.67,7.04,9.03,16.71,22.0,0.73,1.26,25.71,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-14.20,42.0,M-LC,4.26,212994.0,11514.0,15698.0,0.22,5.71,7.37,13.51,60.0,0.73,1.61,19.03,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RAJOOENG,143.33,-32.60,39.0,H-SC,9.98,102000.0,-500.0,41330.0,0.60,-0.49,40.52,39.83,114.0,-0.01,0.77,0.99,AR,ATH,MISC
39,ICICIPRULI,790.00,-20.65,37.0,H-MC,2.37,136560.0,772.0,41979.0,-0.87,0.57,30.74,31.48,107.0,0.02,1.03,12.76,X40,ATH,INSURANCE
38,ICICIGI,2260.25,-20.34,37.0,H-MC,4.91,137055.0,922.0,30207.0,-0.44,0.68,22.04,22.86,91.0,0.03,1.03,13.38,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.30,34.0,H-LC,13.48,241080.0,-20539.0,139754.0,0.44,-7.85,57.97,45.57,7.0,-0.15,1.82,2.58,AR,NTT,ELECTRICAL
47,JIOFIN,387.00,-1.88,38.0,H-LC,15.37,217070.0,-336.0,53833.0,-1.79,-0.15,24.80,24.61,48.0,-0.01,1.64,54.58,XY24,BTT,FINANCE
33,HCLTECH,1943.91,0.22,40.0,H-LC,10.19,229220.0,-12704.0,75964.0,0.70,-5.25,33.14,26.15,8.0,-0.17,1.73,8.32,X40,ATH,IT
87,VBL,671.64,-12.96,43.0,H-LC,8.06,313290.0,-2552.0,115228.0,-1.34,-0.81,36.78,35.67,5.0,-0.02,2.36,12.93,X40N,ATH,FMCG
69,SIEMENS,7969.85,-4.32,45.0,H-LC,32.65,153755.0,-32340.0,244732.0,1.05,-17.38,159.17,114.13,15.0,-0.13,1.16,13.23,AR,ATH,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-11.91,52.0,H-LC,2.82,271229.0,5803.0,50991.0,-0.28,2.19,18.80,21.40,11.0,0.11,2.05,9.68,XY25,NTT,FMCG
51,LTIM,7230.2,-6.94,48.0,H-LC,2.92,231008.0,-15143.0,94344.0,0.21,-6.15,40.84,32.18,16.0,-0.16,1.74,27.67,X200,ATH,IT
45,ITC,452.0,-38.34,48.0,H-LC,3.00,199463.0,-675.0,21562.0,1.75,-0.34,10.81,10.44,4.0,-0.03,1.50,5.21,X40,NTT,FMCG
20,CIPLA,1795.0,-18.75,62.0,H-LC,4.40,217008.0,12508.0,28905.0,0.23,6.12,13.32,20.25,10.0,0.43,1.64,15.38,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.54,56.0,H-LC,7.22,220792.0,-30976.0,153208.0,0.80,-12.30,69.39,48.55,27.0,-0.20,1.66,18.76,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,71.0,L-SC,26.70,88800.0,-24749.0,64797.0,8.72,-21.80,72.97,35.27,268.0,-0.38,0.67,117.34,XR,NTT,HOTELS
60,RELAXO,1176.00,-33.88,57.0,H-SC,8.51,61338.0,-54200.0,89192.0,7.04,-46.91,145.41,30.28,136.0,-0.61,0.46,20.59,X40N,NTT,FOOTWEAR
64,SAMMAANCAP,326.00,-165.91,55.0,M-SC,8.19,75432.0,-26778.0,120171.0,6.57,-26.20,159.31,91.37,208.0,-0.22,0.57,22.45,XY25,NTT,FINANCE
23,COLPAL,3767.14,0.29,58.0,H-MC,10.60,182450.0,-29291.0,111386.0,3.38,-13.83,61.05,38.77,84.0,-0.26,1.38,8.59,XY25,ATH,FMCG
18,CAMS,5211.76,-7.82,42.0,H-SC,6.23,104913.0,2909.0,41021.0,2.44,2.85,39.10,43.06,122.0,0.07,0.79,20.46,X40N,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,JIOFIN,387.0,-1.88,38.0,H-LC,15.37,217070.0,-336.0,53833.0,-1.79,-0.15,24.80,24.61,48.0,-0.01,1.64,54.58,XY24,BTT,FINANCE
85,VAIBHAVGBL,521.0,34.99,35.0,H-SC,12.11,90055.0,-61936.0,136055.0,-1.77,-40.75,151.08,48.77,125.0,-0.46,0.68,4.82,XR,NTT,JEWELLERY
76,TATAELXSI,9161.0,-26.40,18.0,H-MC,6.42,73430.0,-29222.0,54823.0,-1.74,-28.47,74.66,24.94,98.0,-0.53,0.55,11.62,OX40N,NTT,IT
4,APOLLOHOSP,8285.0,-16.30,51.0,H-LC,8.37,167178.0,11001.0,15096.0,-1.67,7.04,9.03,16.71,22.0,0.73,1.26,25.71,X40N,BTT,HEALTHCARE
43,INFY,2275.0,-19.24,45.0,H-LC,11.73,314984.0,2096.0,169587.0,-1.42,0.67,53.84,54.87,3.0,0.01,2.38,7.36,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,100.43,61.0,M-SC,8.16,86329.0,-14448.0,14538.0,1.82,-14.34,16.84,0.09,245.0,-0.99,0.65,13.42,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,41.0,H-SC,14.60,115020.0,-27459.0,46986.0,0.40,-19.27,40.85,13.70,163.0,-0.58,0.87,35.76,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.52,51.0,H-SC,3.45,216513.0,-53154.0,89485.0,1.06,-19.71,41.33,13.47,138.0,-0.59,1.63,9.51,XY24,NTT,PAINTS
19,CERA,9475.0,-20.76,41.0,H-SC,3.03,113220.0,-30583.0,57335.0,0.69,-21.27,50.64,18.60,149.0,-0.53,0.85,24.94,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,108.15,39.0,M-SC,2.19,136363.0,-38196.0,38223.0,0.22,-21.88,28.03,0.02,221.0,-1.00,1.03,18.61,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.76,49.0,H-LC,15.58,291391.0,-54565.0,178419.0,0.20,-15.77,61.23,35.80,1.0,-0.31,2.20,3.23,X40,BTT,IT
43,INFY,2275.00,-19.24,45.0,H-LC,11.73,314984.0,2096.0,169587.0,-1.42,0.67,53.84,54.87,3.0,0.01,2.38,7.36,X40,BTT,IT
45,ITC,452.00,-38.34,48.0,H-LC,3.00,199463.0,-675.0,21562.0,1.75,-0.34,10.81,10.44,4.0,-0.03,1.50,5.21,X40,NTT,FMCG
87,VBL,671.64,-12.96,43.0,H-LC,8.06,313290.0,-2552.0,115228.0,-1.34,-0.81,36.78,35.67,5.0,-0.02,2.36,12.93,X40N,ATH,FMCG
1,ABB,7934.00,-42.30,34.0,H-LC,13.48,241080.0,-20539.0,139754.0,0.44,-7.85,57.97,45.57,7.0,-0.15,1.82,2.58,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,13.12,59.0,H-MC,15.05,161414.0,35214.0,10960.0,-0.60,27.90,6.79,36.59,108.0,3.21,1.22,43.97,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-6.78,74.0,H-LC,12.83,232799.0,48318.0,22046.0,1.09,26.19,9.47,38.14,38.0,2.19,1.76,39.57,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.12,59.0,H-MC,15.05,161414.0,35214.0,10960.0,-0.60,27.90,6.79,36.59,108.0,3.21,1.22,43.97,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.48,59.0,H-MC,4.05,188900.0,24230.0,43730.0,0.47,14.71,23.15,41.28,89.0,0.55,1.42,23.87,X40N,ATH,AC
88,VOLTAS,1918.49,-1.64,60.0,H-MC,6.33,206460.0,14718.0,81304.0,1.26,7.68,39.38,50.08,99.0,0.18,1.56,15.00,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.47,71.0,H-MC,15.85,178535.0,27622.0,29690.0,0.18,18.30,16.63,37.97,93.0,0.93,1.35,48.24,AR,ATH,AUTO
24,DABUR,735.00,-6.72,54.0,H-MC,3.78,203171.0,6143.0,83483.0,1.38,3.12,41.09,45.49,102.0,0.07,1.53,16.56,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.47,71.0,H-MC,15.85,178535.0,27622.0,29690.0,0.18,18.30,16.63,37.97,93.0,0.93,1.35,48.24,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.72,51.0,H-SC,12.48,137747.0,14411.0,119358.0,0.83,11.68,86.65,108.45,119.0,0.12,1.04,36.65,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-41.67,51.0,M-SC,5.59,98475.0,6976.0,71778.0,1.26,7.62,72.89,86.07,223.0,0.10,0.74,43.00,XR,NTT,DURABLES
90,WIPRO,420.00,-11.98,50.0,M-LC,7.73,154702.0,3757.0,105692.0,-0.45,2.49,68.32,72.51,53.0,0.04,1.17,8.66,XR,NTT,IT
59,RAJOOENG,143.33,-32.60,39.0,H-SC,9.98,102000.0,-500.0,41330.0,0.60,-0.49,40.52,39.83,114.0,-0.01,0.77,0.99,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-6.69,36.0,H-SC,16.51,86826.0,-14137.0,108671.0,-0.23,-14.00,125.16,93.63,148.0,-0.13,0.65,29.32,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,71.0,L-SC,26.70,88800.0,-24749.0,64797.0,8.72,-21.80,72.97,35.27,268.0,-0.38,0.67,117.34,XR,NTT,HOTELS
52,MASFIN,398.61,-18.70,48.0,H-SC,12.72,92265.0,-5715.0,27320.0,0.33,-5.83,29.61,22.05,152.0,-0.21,0.70,34.11,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,55.0,H-LC,7.47,112224.0,7912.0,8742.0,-0.61,7.58,7.79,15.96,9.0,0.91,0.85,25.76,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,41.0,H-SC,14.60,115020.0,-27459.0,46986.0,0.40,-19.27,40.85,13.70,163.0,-0.58,0.87,35.76,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,71.0,L-SC,26.70,88800.0,-24749.0,64797.0,8.72,-21.80,72.97,35.27,268.0,-0.38,0.67,117.34,XR,NTT,HOTELS
27,DMART,5201.00,-6.78,74.0,H-LC,12.83,232799.0,48318.0,22046.0,1.09,26.19,9.47,38.14,38.0,2.19,1.76,39.57,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.47,71.0,H-MC,15.85,178535.0,27622.0,29690.0,0.18,18.30,16.63,37.97,93.0,0.93,1.35,48.24,AR,ATH,AUTO
14,BLUESTARCO,2326.38,9.48,59.0,H-MC,4.05,188900.0,24230.0,43730.0,0.47,14.71,23.15,41.28,89.0,0.55,1.42,23.87,X40N,ATH,AC
23,COLPAL,3767.14,0.29,58.0,H-MC,10.60,182450.0,-29291.0,111386.0,3.38,-13.83,61.05,38.77,84.0,-0.26,1.38,8.59,XY25,ATH,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.69
1,25,43.12
2,50,72.72


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.06
LC    36.78
MC    26.18
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.14
X40      15.50
X40N     14.25
XY25      9.80
AR        9.33
XR        8.97
OX40N     7.31
X5K       2.24
X200      1.74
MH        1.67
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.64
H-MC    23.32
H-SC    22.31
M-SC    13.17
M-LC     7.10
M-MC     2.54
L-SC     1.58
L-LC     1.04
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.21,-1.31,34.31
IT,13.36,-16.57,74.38
BANKS,7.34,-17.35,67.18
FINANCE,7.22,-25.95,72.40
PAINTS,5.94,-17.01,43.19
MISC,5.84,-18.47,86.38
ELECTRICAL,5.65,-9.84,70.38
AUTO,4.56,-12.06,60.82
AC,3.71,1.70,31.59


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2905212.0,23
AR,1244451.0,10
XR,1104601.0,13
X40,911330.0,10
X40N,542442.0,11
OX40N,487540.0,10
XY25,444046.0,7
SR,200558.0,2
X5K,142573.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2815663.0,27
M-SC,1499197.0,17
H-LC,1435380.0,17
H-MC,1284939.0,18
M-LC,438866.0,5
M-MC,340510.0,2
L-SC,251351.0,3
L-MC,60042.0,1
L-LC,45757.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,949095.0,6
M-SC,XY24,881795.0,7
H-SC,AR,668911.0,5
H-LC,X40,598740.0,5
H-SC,XR,534250.0,6
H-MC,XY24,496334.0,5
H-LC,AR,384486.0,2
M-MC,XY24,340510.0,2
H-MC,X40,230377.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
